## Notebook for creating Simulation 3

See Section 6.3 of Susobhanan+ 2024.

In [28]:
from pint.models import get_model, get_model_and_toas
from pint.simulation import make_fake_toas_fromtim
from pint.logging import setup as setup_log
from pint.utils import wavex_setup, plrednoise_from_wavex
from pint.fitter import WLSFitter

from io import StringIO
import numpy as np
import astropy.units as u
from copy import deepcopy

from joblib import Parallel, delayed

In [29]:
setup_log(level="WARNING")

2

In [30]:
par_sim = """
    PSR           SIM3
    RAJ           05:00:00     1
    DECJ          15:00:00     1
    PEPOCH        55000
    F0            100          1
    F1            -1e-15       1 
    PHOFF         0            1
    DM            15           1
    TNREDAMP      -13
    TNREDGAM      3.5
    TNREDC        30
    TZRMJD        55000
    TZRFRQ        1400 
    TZRSITE       gbt
    UNITS         TDB
    EPHEM         DE440
    CLOCK         TT(BIPM2019)
"""

In [31]:
m0 = get_model(StringIO(par_sim))

In [32]:
nharm_opt = 17

In [33]:
m1, t = get_model_and_toas("sim3.wx1.par", "sim3.tim")

In [34]:
Tspan = t.get_mjds().max() - t.get_mjds().min()

idxs = m1.components["WaveX"].get_indices()
for idx in reversed(idxs):
    if idx > nharm_opt:
        m1.components["WaveX"].remove_wavex_component(idx)

ftr = WLSFitter(t, m1)
ftr.fit_toas(maxiter=5)

2034.3447990774315404

In [35]:
print(ftr.model)

# Created: 2024-05-27T16:23:39.235470
# PINT_version: 1.0+226.g659c08ce
# User: Abhimanyu Susobhanan (abhimanyu)
# Host: abhimanyu-HP-Envy-x360-2-in-1-Laptop-15-fh0xxx
# OS: Linux-6.5.0-35-generic-x86_64-with-glibc2.35
# Python: 3.12.0 | packaged by Anaconda, Inc. | (main, Oct  2 2023, 17:29:18) [GCC 11.2.0]
# Format: pint
PSR                                  SIM3
EPHEM                               DE440
CLOCK                        TT(BIPM2019)
UNITS                                 TDB
START              53000.9999999566487616
FINISH             56985.0000000463726042
DILATEFREQ                              N
DMDATA                                  N
NTOA                                 2000
CHI2                   2034.3447990774316
CHI2R                  1.0384608469001693
TRES                1.0166290032666464267
RAJ                      5:00:00.00017315 1 0.00011139189982086978
DECJ                    14:59:59.98792426 1 0.01183671859058394935
PMRA                                 

In [41]:
def simulate_and_measure():
    setup_log(level="WARNING")
    
    t = make_fake_toas_fromtim(
        "sim3.tim",
        m0,
        add_noise=True,
        add_correlated_noise=True,
        name="fake",
    )

    try:
        ftr = WLSFitter(t, m1)
        ftr.fit_toas(maxiter=6)
    
        m_pint_1 = plrednoise_from_wavex(ftr.model)
    
        print(
            m_pint_1.TNREDAMP.value,
            m_pint_1.TNREDGAM.value
        )
        
        return (
            m_pint_1.TNREDAMP.value,
            m_pint_1.TNREDGAM.value
        )
    except:
        return np.nan, np.nan

In [42]:
results = Parallel(n_jobs=8)(delayed(simulate_and_measure)() for i in range(500))

KeyboardInterrupt: 

In [24]:
# Lines commented out to avoid accidentally rewriting the files. 

# m.write_parfile("simulations/sim3.par")
# t.write_TOA_file("simulations/sim3.tim")

In [25]:
m1 = deepcopy(m)
m1.remove_component("PLRedNoise")

Tspan = t.get_mjds().max() - t.get_mjds().min()
wavex_setup(m1, Tspan, n_freqs=45)

# Lines commented out to avoid accidentally rewriting the files.
# m1.write_parfile("simulations/sim3.wx.par")